In [ ]:
pip install Afinn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Afinn: filename=afinn-0.1-py3-none-any.whl size=53448 sha256=834cb1070adb7b7c471ddc298099960121ca66b7ad14d066484685ee24769288
  Stored in directory: /root/.cache/pip/wheels/79/91/ee/8374d9bc8c6c0896a2db75afdfd63d43653902407a0e76cd94
Successfully built Afinn


In [ ]:
# import needed libraries
import pandas as pd 


import os 
import re
import nltk
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk.corpus import stopwords
nltk.download('wordnet')
from bs4 import BeautifulSoup
nltk.download('stopwords')

from gensim.models.word2vec import Word2Vec
import numpy as np
import csv
import urllib.request
import math

from afinn import Afinn
from nltk.corpus import sentiwordnet as swn
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet as wn
nltk.download('sentiwordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.


True

In [ ]:
#Get all the data from the url

#url = 'https://raw.githubusercontent.com/sareek/NLP_on_Social_Media_Data/sareek/data/project_data.2023-03-11_13.11.55.783862.json' #In json format
url = 'https://raw.githubusercontent.com/sareek/NLP_on_Social_Media_Data/sareek/data/project_data.2023-03-11_13.11.55.783862.csv'   #In csv format

df = pd.read_csv(url, sep = ',', index_col = 0)
duplicated_rows = df.duplicated().sum()
combined_df = df.drop_duplicates()

print(df.size)
print(combined_df.size)

768201
678111


<ipython-input-3-ebd05306dbf7>:6: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, sep = ',', index_col = 0)


In [ ]:
combined_df.head()

,conversation_id,lang,reply_settings,created_at,text,author_id,referenced_tweets,id,edit_history_tweet_ids,public_metrics.retweet_count,...,public_metrics.like_count,public_metrics.quote_count,public_metrics.impression_count,in_reply_to_user_id,geo.place_id,withheld.copyright,withheld.country_codes,keyword,geo.coordinates.type,geo.coordinates.coordinates
0,1633954063934009344,en,everyone,2023-03-09 22:13:00+00:00,RT @Infantry_bucky: He’s lucky a #ChrisRockLiv...,1519164980582653952,"[{'type': 'retweeted', 'id': '1633938373529292...",1633954063934009344,['1633954063934009354'],8,...,0,0,0,NaN,NaN,NaN,NaN,#chrisrocklive,NaN,NaN
1,1633954058212876288,en,everyone,2023-03-09 22:12:59+00:00,RT @1_ofakindnocap: Chris Rock: “we all been c...,21575184,"[{'type': 'retweeted', 'id': '1632283297588948...",1633954058212876288,['1633954058212876289'],616,...,0,0,0,NaN,NaN,NaN,NaN,#chrisrocklive,NaN,NaN
2,1633951267423768576,en,everyone,2023-03-09 22:01:54+00:00,RT @1_ofakindnocap: Chris Rock: “we all been c...,360633018,"[{'type': 'retweeted', 'id': '1632283297588948...",1633951267423768576,['1633951267423768578'],616,...,0,0,0,NaN,NaN,NaN,NaN,#chrisrocklive,NaN,NaN
3,1633950853626318848,en,everyone,2023-03-09 22:00:15+00:00,RT @rolandsmartin: Working out and watching th...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632825595473149...",1633950853626318848,['1633950853626318850'],14,...,0,0,0,NaN,NaN,NaN,NaN,#chrisrocklive,NaN,NaN
4,1633950824664645632,en,everyone,2023-03-09 22:00:08+00:00,RT @rolandsmartin: Workout done. I’ll have a f...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632833853021728...",1633950824664645632,['1633950824664645634'],8,...,0,0,0,NaN,NaN,NaN,NaN,#chrisrocklive,NaN,NaN


In [ ]:
def clean_tweets(tweets):
    # Stopword removal, converting uppercase into lower case, and lemmatization
    stopwords = nltk.corpus.stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    nltk.download('stopwords')
    data_without_stopwords = []
    for i in range(0, len(tweets)):
        doc = re.sub('[^a-zA-Z]', ' ' , tweets[i])
        doc = doc.lower()
        doc = doc.split()
        doc = [lemmatizer.lemmatize(word) for word in doc if not word in set(stopwords)]
        doc = ' '.join(doc)
        data_without_stopwords.append(doc)
        
    return data_without_stopwords


#get tweets from dataframe
tweets = [i for i in combined_df["text"]]
cleaned_tweets = clean_tweets(tweets)
#add clean_text column with list of clean text
combined_df['clean_text'] = cleaned_tweets

#optional step just reorderning
new_order = [
    'conversation_id', 
    'lang', 
    'reply_settings', 
    'created_at', 
    'clean_text',
    'text',
    'author_id',
    'referenced_tweets',
    'id',
    'edit_history_tweet_ids',
    'public_metrics.retweet_count',
    'public_metrics.reply_count',
    'public_metrics.like_count',
    'public_metrics.impression_count',
    'in_reply_to_user_id',
    'geo.place_id',
    'withheld.copyright',
    'withheld.country_codes',
    'geo.coordinates.type',
    'geo.coordinates.coordinates'
]
combined_df = combined_df.reindex(columns=new_order)

combined_df.head()
#Statistics on the data, mean, median, std deviation, etc.
#combined_df.describe()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-5-bed31827238c>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df['clean_text'] = cleaned_tweets


,conversation_id,lang,reply_settings,created_at,clean_text,text,author_id,referenced_tweets,id,edit_history_tweet_ids,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.impression_count,in_reply_to_user_id,geo.place_id,withheld.copyright,withheld.country_codes,geo.coordinates.type,geo.coordinates.coordinates
0,1633954063934009344,en,everyone,2023-03-09 22:13:00+00:00,rt infantry bucky lucky chrisrocklive worse th...,RT @Infantry_bucky: He’s lucky a #ChrisRockLiv...,1519164980582653952,"[{'type': 'retweeted', 'id': '1633938373529292...",1633954063934009344,['1633954063934009354'],8,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1633954058212876288,en,everyone,2023-03-09 22:12:59+00:00,rt ofakindnocap chris rock cheated never perso...,RT @1_ofakindnocap: Chris Rock: “we all been c...,21575184,"[{'type': 'retweeted', 'id': '1632283297588948...",1633954058212876288,['1633954058212876289'],616,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,1633951267423768576,en,everyone,2023-03-09 22:01:54+00:00,rt ofakindnocap chris rock cheated never perso...,RT @1_ofakindnocap: Chris Rock: “we all been c...,360633018,"[{'type': 'retweeted', 'id': '1632283297588948...",1633951267423768576,['1633951267423768578'],616,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,1633950853626318848,en,everyone,2023-03-09 22:00:15+00:00,rt rolandsmartin working watching chrisrock ne...,RT @rolandsmartin: Working out and watching th...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632825595473149...",1633950853626318848,['1633950853626318850'],14,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,1633950824664645632,en,everyone,2023-03-09 22:00:08+00:00,rt rolandsmartin workout done comment chrisroc...,RT @rolandsmartin: Workout done. I’ll have a f...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632833853021728...",1633950824664645632,['1633950824664645634'],8,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


### AFINN Model

In [ ]:
af = Afinn()

# compute sentiment scores (polarity) and labels
sentiment_scores = [af.score(article) for article in combined_df['clean_text']]
sentiment_category = ['positive' if score > 0 
                          else 'negative' if score < 0 
                              else 'neutral' 
                                  for score in sentiment_scores]
    
    
# sentiment statistics per tweet category
df = pd.DataFrame()
df['text'] =  combined_df['clean_text']
df['scores'] = sentiment_scores
df['sentiments'] = sentiment_category
print(df)


                                                    text  scores sentiments
0      rt infantry bucky lucky chrisrocklive worse th...    -2.0   negative
1      rt ofakindnocap chris rock cheated never perso...    -6.0   negative
2      rt ofakindnocap chris rock cheated never perso...    -6.0   negative
3      rt rolandsmartin working watching chrisrock ne...     0.0    neutral
4      rt rolandsmartin workout done comment chrisroc...     0.0    neutral
...                                                  ...     ...        ...
36576  rt ministerboomer way susannem maybe take deto...     0.0    neutral
36577  russia ukraine belarus occupier planning large...     0.0    neutral
36578  rt kajakallas year ago soviet plane bombed tal...    -6.0   negative
36579  rt sternenko today civilian killed russian khe...    -2.0   negative
36580  rt sunshinelove u lose six nuclear weapon piss...    -6.0   negative

[32291 rows x 3 columns]


In [ ]:
df.head()

,text,scores,sentiments
0,rt infantry bucky lucky chrisrocklive worse th...,-2.0,negative
1,rt ofakindnocap chris rock cheated never perso...,-6.0,negative
2,rt ofakindnocap chris rock cheated never perso...,-6.0,negative
3,rt rolandsmartin working watching chrisrock ne...,0.0,neutral
4,rt rolandsmartin workout done comment chrisroc...,0.0,neutral


In [ ]:
df.to_csv('AFINN.csv')

### SentiWordNet

In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
# POS tagger dictionary
pos_dict = {'J':wn.ADJ, 'V':wn.VERB, 'N':wn.NOUN, 'R':wn.ADV}

#tokenization, pos_tagging and stop words removal
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('english')):
          newlist.append(tuple([word, pos_dict.get(tag[0])]))
    return newlist

combined_df['POS tagged'] = combined_df['clean_text'].apply(token_stop_pos)
combined_df.head()

,conversation_id,lang,reply_settings,created_at,clean_text,text,author_id,referenced_tweets,id,edit_history_tweet_ids,...,public_metrics.reply_count,public_metrics.like_count,public_metrics.impression_count,in_reply_to_user_id,geo.place_id,withheld.copyright,withheld.country_codes,geo.coordinates.type,geo.coordinates.coordinates,POS tagged
0,1633954063934009344,en,everyone,2023-03-09 22:13:00+00:00,rt infantry bucky lucky chrisrocklive worse th...,RT @Infantry_bucky: He’s lucky a #ChrisRockLiv...,1519164980582653952,"[{'type': 'retweeted', 'id': '1633938373529292...",1633954063934009344,['1633954063934009354'],...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,"[(rt, n), (infantry, n), (bucky, a), (lucky, a..."
1,1633954058212876288,en,everyone,2023-03-09 22:12:59+00:00,rt ofakindnocap chris rock cheated never perso...,RT @1_ofakindnocap: Chris Rock: “we all been c...,21575184,"[{'type': 'retweeted', 'id': '1632283297588948...",1633954058212876288,['1633954058212876289'],...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,"[(rt, n), (ofakindnocap, None), (chris, v), (r..."
2,1633951267423768576,en,everyone,2023-03-09 22:01:54+00:00,rt ofakindnocap chris rock cheated never perso...,RT @1_ofakindnocap: Chris Rock: “we all been c...,360633018,"[{'type': 'retweeted', 'id': '1632283297588948...",1633951267423768576,['1633951267423768578'],...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,"[(rt, n), (ofakindnocap, None), (chris, v), (r..."
3,1633950853626318848,en,everyone,2023-03-09 22:00:15+00:00,rt rolandsmartin working watching chrisrock ne...,RT @rolandsmartin: Working out and watching th...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632825595473149...",1633950853626318848,['1633950853626318850'],...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,"[(rt, n), (rolandsmartin, n), (working, v), (w..."
4,1633950824664645632,en,everyone,2023-03-09 22:00:08+00:00,rt rolandsmartin workout done comment chrisroc...,RT @rolandsmartin: Workout done. I’ll have a f...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632833853021728...",1633950824664645632,['1633950824664645634'],...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,"[(rt, n), (rolandsmartin, n), (workout, None),..."


In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
      if not pos:
        lemma = word
        lemma_rew = lemma_rew + " " + lemma
      else:
        lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
        lemma_rew = lemma_rew + " " + lemma
    return lemma_rew

combined_df['Lemma'] = combined_df['POS tagged'].apply(lemmatize)
combined_df.head()

,conversation_id,lang,reply_settings,created_at,clean_text,text,author_id,referenced_tweets,id,edit_history_tweet_ids,...,public_metrics.like_count,public_metrics.impression_count,in_reply_to_user_id,geo.place_id,withheld.copyright,withheld.country_codes,geo.coordinates.type,geo.coordinates.coordinates,POS tagged,Lemma
0,1633954063934009344,en,everyone,2023-03-09 22:13:00+00:00,rt infantry bucky lucky chrisrocklive worse th...,RT @Infantry_bucky: He’s lucky a #ChrisRockLiv...,1519164980582653952,"[{'type': 'retweeted', 'id': '1633938373529292...",1633954063934009344,['1633954063934009354'],...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,"[(rt, n), (infantry, n), (bucky, a), (lucky, a...",rt infantry bucky lucky chrisrocklive bad th...
1,1633954058212876288,en,everyone,2023-03-09 22:12:59+00:00,rt ofakindnocap chris rock cheated never perso...,RT @1_ofakindnocap: Chris Rock: “we all been c...,21575184,"[{'type': 'retweeted', 'id': '1632283297588948...",1633954058212876288,['1633954058212876289'],...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,"[(rt, n), (ofakindnocap, None), (chris, v), (r...",rt ofakindnocap chris rock cheat never perso...
2,1633951267423768576,en,everyone,2023-03-09 22:01:54+00:00,rt ofakindnocap chris rock cheated never perso...,RT @1_ofakindnocap: Chris Rock: “we all been c...,360633018,"[{'type': 'retweeted', 'id': '1632283297588948...",1633951267423768576,['1633951267423768578'],...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,"[(rt, n), (ofakindnocap, None), (chris, v), (r...",rt ofakindnocap chris rock cheat never perso...
3,1633950853626318848,en,everyone,2023-03-09 22:00:15+00:00,rt rolandsmartin working watching chrisrock ne...,RT @rolandsmartin: Working out and watching th...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632825595473149...",1633950853626318848,['1633950853626318850'],...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,"[(rt, n), (rolandsmartin, n), (working, v), (w...",rt rolandsmartin work watch chrisrock netfli...
4,1633950824664645632,en,everyone,2023-03-09 22:00:08+00:00,rt rolandsmartin workout done comment chrisroc...,RT @rolandsmartin: Workout done. I’ll have a f...,1547101103803830272,"[{'type': 'retweeted', 'id': '1632833853021728...",1633950824664645632,['1633950824664645634'],...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,"[(rt, n), (rolandsmartin, n), (workout, None),...",rt rolandsmartin workout do comment chrisroc...


In [ ]:
combined_df[['clean_text', 'Lemma']]

,clean_text,Lemma
0,rt infantry bucky lucky chrisrocklive worse th...,rt infantry bucky lucky chrisrocklive bad th...
1,rt ofakindnocap chris rock cheated never perso...,rt ofakindnocap chris rock cheat never perso...
2,rt ofakindnocap chris rock cheated never perso...,rt ofakindnocap chris rock cheat never perso...
3,rt rolandsmartin working watching chrisrock ne...,rt rolandsmartin work watch chrisrock netfli...
4,rt rolandsmartin workout done comment chrisroc...,rt rolandsmartin workout do comment chrisroc...
...,...,...
36576,rt ministerboomer way susannem maybe take deto...,rt ministerboomer way susannem maybe take de...
36577,russia ukraine belarus occupier planning large...,russia ukraine belarus occupier plan large s...
36578,rt kajakallas year ago soviet plane bombed tal...,rt kajakallas year ago soviet plane bomb tal...
36579,rt sternenko today civilian killed russian khe...,rt sternenko today civilian kill russian khe...


In [ ]:
pos=neg=obj=count=0

postagging = []

for review in combined_df['Lemma']:
    list = word_tokenize(review)
    postagging.append(nltk.pos_tag(list))

combined_df['POS tagged'] = postagging

def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None


# Returns list of pos-neg and objective score. But returns empty list if not present in senti wordnet.
def get_sentiment(word,tag):
    wn_tag = penn_to_wn(tag)
    
    if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
        return []

    #Lemmatization
    lemma = wordnet_lemmatizer.lemmatize(word, pos=wn_tag)
    if not lemma:
        return []

    #Synset is a special kind of a simple interface that is present in NLTK to look up words in WordNet. 
    #Synset instances are the groupings of synonymous words that express the same concept. 
    #Some of the words have only one Synset and some have several.
    synsets = wn.synsets(word, pos=wn_tag)
    if not synsets:
      return []

    # Take the first sense, the most common
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())

    return [synset.name(), swn_synset.pos_score(),swn_synset.neg_score(),swn_synset.obj_score()]

    pos=neg=obj=count=0

In [ ]:
nltk.download('sentiwordnet')

[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [ ]:
senti_score = []

for pos_val in combined_df['POS tagged']:
    senti_val = [get_sentiment(x,y) for (x,y) in pos_val]
    for score in senti_val:
        try:
            pos = pos + score[1]  #positive score is stored at 2nd position
            neg = neg + score[2]  #negative score is stored at 3rd position
        except:
            continue
    senti_score.append(pos - neg)
    pos=neg=0    
    
combined_df['senti_score'] = senti_score
print(combined_df['senti_score'])

print(combined_df.head)

0        0.250
1       -0.625
2       -0.625
3        0.000
4        0.000
         ...  
36576    0.250
36577    0.000
36578   -1.125
36579    0.125
36580    0.500
Name: senti_score, Length: 32291, dtype: float64
<bound method NDFrame.head of            conversation_id lang reply_settings                 created_at  \
0      1633954063934009344   en       everyone  2023-03-09 22:13:00+00:00   
1      1633954058212876288   en       everyone  2023-03-09 22:12:59+00:00   
2      1633951267423768576   en       everyone  2023-03-09 22:01:54+00:00   
3      1633950853626318848   en       everyone  2023-03-09 22:00:15+00:00   
4      1633950824664645632   en       everyone  2023-03-09 22:00:08+00:00   
...                    ...  ...            ...                        ...   
36576  1633873450594091008   en       everyone  2023-03-09 16:52:41+00:00   
36577  1633873450422140928   en       everyone  2023-03-09 16:52:41+00:00   
36578  1633873447897149440   en       everyone  2023-03-09 16:5

In [ ]:
df1 = pd.DataFrame()
df1['text'] =  combined_df['clean_text']
df1['scores'] = combined_df['senti_score']
#df['sentiments'] = sentiment_category
print(df1)

                                                    text  scores
0      rt infantry bucky lucky chrisrocklive worse th...   0.250
1      rt ofakindnocap chris rock cheated never perso...  -0.625
2      rt ofakindnocap chris rock cheated never perso...  -0.625
3      rt rolandsmartin working watching chrisrock ne...   0.000
4      rt rolandsmartin workout done comment chrisroc...   0.000
...                                                  ...     ...
36576  rt ministerboomer way susannem maybe take deto...   0.250
36577  russia ukraine belarus occupier planning large...   0.000
36578  rt kajakallas year ago soviet plane bombed tal...  -1.125
36579  rt sternenko today civilian killed russian khe...   0.125
36580  rt sunshinelove u lose six nuclear weapon piss...   0.500

[32291 rows x 2 columns]


In [ ]:
df1.to_csv('SentiWordNet.csv')

In [ ]:
#Range

# overall=[]
# for i in range(len(combined_df)):
#     if combined_df['senti_score'][i]>= 0.05:
#         overall.append('Positive')
#     elif combined_df['senti_score'][i]<= -0.05:
#         overall.append('Negative')
#     else:
#         overall.append('Neutral')
# combined_df['Overall Sentiment']=overall

In [ ]:
#data_source_url = "https://raw.githubusercontent.com/sareek/NLP_on_Social_Media_Data/main/data/%23chrisrocklive_project_tweets.2023-03-09_17.16.18.676751.json"
tweets = pd.read_csv('anu_annotations.tsv')

In [ ]:
tweets

,Unnamed: 0.1,Unnamed: 0,conversation_id,lang,reply_settings,created_at,clean_text,text,author_id,referenced_tweets,...,public_metrics.reply_count,public_metrics.like_count,public_metrics.impression_count,in_reply_to_user_id,geo.place_id,withheld.copyright,withheld.country_codes,geo.coordinates.type,geo.coordinates.coordinates,our_label
0,26657,30947,1633209504631513088,en,everyone,2023-03-07 20:54:24+00:00,rt hegelwcrmcheese fuss senate unequal represe...,RT @HegelwCrmCheese: All the fuss about the Se...,14794913,"[{'type': 'retweeted', 'id': '1633174877141184...",...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12749,17039,1633668866172649472,en,everyone,2023-03-09 03:19:44+00:00,people know schumer admitted communist earlier...,Most people don't know that Schumer was an adm...,1494696078553325568,"[{'type': 'quoted', 'id': '1633598307179335680'}]",...,0,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14003,18293,1633661305801326592,en,everyone,2023-03-09 02:56:49+00:00,msavaarmstrong coarse another extension obama ...,@MsAvaArmstrong Of coarse... another extension...,1413055458290683904,"[{'type': 'replied_to', 'id': '163366130580132...",...,0,0,33,2.449914e+09,NaN,NaN,NaN,NaN,NaN,NaN
3,26189,30479,1633345860300947456,en,everyone,2023-03-08 05:56:13+00:00,rt letifayimamu majority gurage zone voter pas...,RT @LetifaYimamu: The majority of Gurage Zone ...,1627483662726864896,"[{'type': 'retweeted', 'id': '1633341568261169...",...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22298,26588,1633933766338179072,en,everyone,2023-03-09 20:52:21+00:00,rt democracydocket c statehood way ensure amer...,RT @DemocracyDocket: “D.C. statehood is the on...,180237277,"[{'type': 'retweeted', 'id': '1633625994052399...",...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,30064,34354,1633875207789359104,en,everyone,2023-03-09 16:59:40+00:00,rt weaponsmonitor least recorded incident expl...,RT @WeaponsMonitor: There were at least 34 rec...,2966707714,"[{'type': 'retweeted', 'id': '1633769804413784...",...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,186,186,1633724515212742656,en,everyone,2023-03-09 07:00:52+00:00,rt netflix chrisrocklive tonight pm et pm pt m...,RT @netflix: #ChrisRockLive TONIGHT at 10pm ET...,2348520354,"[{'type': 'retweeted', 'id': '1631421624359739...",...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,30557,34847,1633874812945694720,en,everyone,2023-03-09 16:58:05+00:00,rt al awkalisasha apocalypseos tweeted ago wai...,RT @Al3awkaliSasha: @apocalypseos I tweeted th...,84140874,"[{'type': 'retweeted', 'id': '1633427966834225...",...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,21637,25927,1633669429819998208,en,everyone,2023-03-09 03:21:58+00:00,rt peterbakernyt trump tell hannity radio show...,RT @peterbakernyt: Trump tells Hannity on his ...,3015958781,"[{'type': 'retweeted', 'id': '1633604224843120...",...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print(len(pd.unique(tweets['clean_text'])))


82


In [ ]:
labels = ['n', '']

References:
1. AFINN: The AFINN lexicon assigns words with a score that runs between -5 and 5, with negative scores indicating negative sentiment and positive scores indicating positive sentiment.
https://www.geeksforgeeks.org/python-sentiment-analysis-using-affin/
https://www.kdnuggets.com/2018/08/emotion-sentiment-analysis-practitioners-guide-nlp-5.html

2. SentiWordNet: https://www.kaggle.com/code/hamishisham/sentiment-analysis-using-sentiwordnet/notebook
